<a href="https://colab.research.google.com/github/johyunkang/adp_certificate/blob/main/stats_textbook_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 7 통계학과 머신러닝 

## 7.1 머신러닝 기본

## 7.2 정규화와 리지 회귀, 라소 회귀 